In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json 
client = bigquery.Client()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

env: GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json


In [31]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [32]:
%%bigquery metadata --project masterarbeit-245718 --verbose 
SELECT * FROM `bigquery-public-data`.crypto_ethereum.INFORMATION_SCHEMA.COLUMNS where table_name = "traces"

Executing query with job ID: 322a332c-e520-4136-83ab-52b15482b0f5
Query executing: 1.03s
Query complete after 1.75s


In [33]:
size_sample_data = 1000

### Generate sample data for "call_type"

In [34]:
# %%bigquery call_types_sql_result --project masterarbeit-245718 --verbose 
# select DISTINCT call_type from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [35]:
call_types = ['call', 'delegatecall', 'staticcall', 'callcode', None]
prob_call_types = [0.7, 0.05, 0.025, 0.025, 0.2]

In [36]:
call_type_sample = np.random.choice(call_types, size_sample_data, p=prob_call_types)

### Generate sample data for "status"

In [37]:
# %%bigquery status_values --project masterarbeit-245718 --verbose 
# select DISTINCT status from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [38]:
status_values = [0, 1]
probs_status_values = [0.05, 0.95]

In [39]:
probs_status_values = [0.05, 0.95]

In [40]:
status_sample = np.random.choice(status_values, size_sample_data, p=probs_status_values)

### Generate sample accounts 

In [41]:
exchanges = ["exchange_{}".format(i) for i in range(1,int(0.05*size_sample_data + 1))]

In [42]:
speculators = ["speculator_{}".format(i) for i in range(1,int(0.95*size_sample_data + 1))]

In [43]:
addresses_testdata = []
addresses_testdata.extend(speculators)
addresses_testdata.extend(exchanges)

In [44]:
addresses_testdata = pd.DataFrame(addresses_testdata, columns=["address"])

## Upload 'addresses' table to bigquery 

In [45]:
addresses_testdata.to_gbq('ethereum_us.addresses_testdata', if_exists="replace")

1it [00:03,  3.79s/it]


### Generate sample transactions (speculators to exchanges)

In [46]:
from_spec_addresses = [np.random.choice(speculators) for i in range(int(size_sample_data/2))]
to_ex_addresses = [np.random.choice(exchanges) for i in range(int(size_sample_data/2))]
values_spec_to_ex = np.random.randint(1, 20, int(size_sample_data/2))

In [47]:
txdata1 = pd.DataFrame(zip(from_spec_addresses, to_ex_addresses, values_spec_to_ex), columns=["from_address", "to_address", "value"])

### Generate sample transactions (exchanges to speculators)

In [48]:
to_spec_addresses = [np.random.choice(speculators) for i in range(int(size_sample_data/2))]
from_ex_addresses = [np.random.choice(exchanges) for i in range(int(size_sample_data/2))]
values_ex_to_spec = np.random.randint(1, 5, int(size_sample_data/2))

In [49]:
txdata2 = pd.DataFrame(zip(from_ex_addresses, to_spec_addresses, values_ex_to_spec), columns=["from_address", "to_address", "value"])

Anmerkung: Die speculators schicken mehr Geld zu den Börsen als umgekehrt.

In [50]:
txdata = txdata1.append(txdata2).reset_index(drop=True)

### Generate sample "block_timestamps"

In [51]:
import datetime as datetime

base = datetime.datetime.utcnow()
block_timestamps = [base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)]

### Merge data to sample "traces" table

In [52]:
traces_sampleData = txdata.copy()

In [53]:
traces_sampleData["status"] = status_sample 

In [54]:
traces_sampleData["call_type"] = call_type_sample 

In [55]:
traces_sampleData["block_timestamp"] = block_timestamps 

In [56]:
traces_sampleData.head()

,from_address,to_address,value,status,call_type,block_timestamp
0,speculator_214,exchange_47,15,1,call,2020-02-03 15:11:31.146042
1,speculator_14,exchange_10,10,1,call,2020-02-03 15:11:19.146042
2,speculator_477,exchange_35,13,1,None,2020-02-03 15:11:07.146042
3,speculator_294,exchange_23,13,1,call,2020-02-03 15:10:55.146042
4,speculator_455,exchange_49,1,1,call,2020-02-03 15:10:43.146042


### Upload "traces" table to bigquery

In [57]:
traces_sampleData.to_gbq('ethereum_us.traces_sampleData', if_exists="replace")

1it [00:03,  3.89s/it]


### Test SQL command for retrieving "weiReceived", "weiSent"

In [58]:
%%bigquery res1 --project masterarbeit-245718 --verbose 
with weiView as (

  with weiReceivedView as (
        
      -- debits
      select to_address, sum(ifnull(value, 0)) as weiReceived
      from `ethereum_us.traces_sampleData` 
      where to_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
        
  ), weiSentView as (
  
      -- credits
      select from_address, sum(ifnull(value, 0)) as weiSent
      from  `ethereum_us.traces_sampleData` 
      where from_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
  ) 
  select 
  CASE 
    when to_address is not null then to_address
    when from_address is not null then from_address
  end as address, 
  ifnull(weiReceived,0) as weiReceived, 
  ifnull(weiSent,0) as weiSent
  from weiReceivedView full outer join weiSentView on from_address = to_address
) 
select address, weiReceived, weiSent from weiView right join `ethereum_us.addresses_testdata`  using(address)

Executing query with job ID: e63669f8-095b-4ae8-883b-bb108e11034c
Query executing: 1.33s
Query complete after 2.25s


In [59]:
res2 = res1.copy()
res2["balance"] = res1.weiReceived - res1.weiSent
res2 = res2.set_index("address")
res2 = res2.sort_values(by="address", ascending=False)
res2 = res2.astype("float")
res2 = res2.fillna(0.)
balance_result_sql = res2

In [60]:
%%bigquery traces_sampleData --project masterarbeit-245718 --verbose 
select * from `ethereum_us.traces_sampleData`

Executing query with job ID: 535cee5a-63f8-4756-abc6-9ae48373c204
Query executing: 0.85s
Query complete after 1.88s


In [61]:
data1 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
data1 = pd.DataFrame(data1)

In [68]:
data2 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data2["weiReceived"] = data1.groupby('to_address').sum().value
data2["weiSent"] = data1.groupby('from_address').sum().value
data2["weiSent"] = data2["weiSent"].fillna(0.)
data2["weiReceived"] = data2["weiReceived"].fillna(0.)
data2["balance"] = data2["weiReceived"] - data2["weiSent"]
data2.index = data2.index.rename("address")
data2 = data2.reindex(addresses_testdata["address"], fill_value=0.)
data2 = data2.sort_values(by="address", ascending=False)
balance_result_py = data2

In [69]:
balance_result_py.head()
balance_result_sql.head()

,weiReceived,weiSent,balance
address,,,
speculator_99,3.0,25.0,-22.0
speculator_98,7.0,0.0,7.0
speculator_97,1.0,0.0,1.0
speculator_96,0.0,3.0,-3.0
speculator_950,0.0,0.0,0.0


,weiReceived,weiSent,balance
address,,,
speculator_99,3.0,25.0,-22.0
speculator_98,7.0,0.0,7.0
speculator_97,1.0,0.0,1.0
speculator_96,0.0,3.0,-3.0
speculator_950,0.0,0.0,0.0


In [70]:
pd.testing.assert_frame_equal(balance_result_py, balance_result_sql)
print("weiSent, weiReceived Test succeeded!!")

weiSent, weiReceived Test succeeded!!


### Test SQL command for retrieving "txSent" and "txReceived"

In [71]:
%%bigquery res2 --project masterarbeit-245718 --verbose 
with txView as (

  with txSent as (
      SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
    ), txReceived as (
      SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
    ) 
    SELECT 
    CASE  
      WHEN to_address IS NOT NULL THEN to_address
      WHEN from_address IS NOT NULL THEN from_address
    END AS address,
    IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
    IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
    from txReceived FULL OUTER JOIN txSent on to_address = from_address
) 

    select address, numberOfTranscationsReceived, numberOfTranscationsSent from txView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 346082c3-4e62-4ba9-b6f0-4b99b73204b0
Query executing: 1.49s
Query complete after 2.50s


In [72]:
res3 = res2.copy()
res3 = res3.fillna(0.)
res3 = res3.sort_values(by="address", ascending=False)
tx_sent_received_result_sql = res3.set_index("address", drop=True)

In [74]:
data3 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data3["numberOfTranscationsReceived"] = data1.groupby('to_address').count().value
data3["numberOfTranscationsSent"] = data1.groupby('from_address').count().value
data3 = data3.fillna(0)
data3 = data3.astype("int")
data3.index = data3.index.rename("address")
data3 = data3.reindex(addresses_testdata["address"], fill_value=0.)
data3 = data3.sort_values(by="address", ascending=False)
tx_sent_received_result_py = data3

In [75]:
pd.testing.assert_frame_equal(tx_sent_received_result_py, tx_sent_received_result_sql)
print("txSent, txReceived Test succeeded !!")

txSent, txReceived Test succeeded !!


### Test avg time between tx

In [76]:
%%bigquery res4 --project masterarbeit-245718 --verbose 
with timeRecView as (

  with receivedTx as (
    SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address),
  
  timeStampDiffs as (
    SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
  
  ) select to_address as address, 
  CASE 
    when (numberOfTranscationsReceived - 1)  > 0 then timestampDiff / (numberOfTranscationsReceived - 1) 
    else 0
  end as avgTimeDiffBetweenReceivedTransactions
  from receivedTx inner join  timeStampDiffs using(to_address)
)

select address, ifnull(avgTimeDiffBetweenReceivedTransactions,0) as avgTimeDiffBetweenReceivedTransactions from timeRecView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 09fe94bc-a8be-4ee6-8a0c-62cbb06a9c77
Query executing: 1.38s
Query complete after 2.31s


In [77]:
res6 = res4.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_receivedtx_result_sql = res6

In [78]:
%%bigquery res5 --project masterarbeit-245718 --verbose 
with timeSentView as (

  with sentTx as (
    SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address),
  timeStampDiffs as (
    SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
  ) select from_address as address, 
  CASE 
    when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
    else 0
  end as avgTimeDiffBetweenSentTransactions
     from sentTx inner join  timeStampDiffs using(from_address)
)

select address, ifnull(avgTimeDiffBetweenSentTransactions,0) as avgTimeDiffBetweenSentTransactions from timeSentView right join `ethereum_us.addresses_testdata` using(address)


with timeSentView as (

  with sentTx as (
    SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address),
  timeStampDiffs as (
    SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
  ) select from_address as address, 
  CASE 
    when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
    else 0
  end as avgTimeDiffBetweenSentTransactions
     from sentTx inner join  timeStampDiffs using(from_address)
)

select address, avgTimeDiffBetweenSentTransactions from timeSentView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 5fd3a20c-ebde-4e9f-9e46-a650ec10202c
Query executing: 1.45s
Query complete after 2.28s


In [357]:
res6 = res5.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_senttx_result_sql = res6

In [360]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res8 = res7.groupby("from_address").max().block_timestamp
res9 = res7.groupby("from_address").min().block_timestamp
res10 = res8 - res9
res10 = res10.rename("seconds_diff")
res10 = tx_sent_received_result_py.join(res10, how="right").drop("numberOfTranscationsReceived", axis=1)
res10 = res10.fillna(0.)
res10["avgTimeDiffBetweenSentTransactions"] = res10["seconds_diff"] / (res10["numberOfTranscationsSent"] - 1)
res10["avgTimeDiffBetweenSentTransactions"] = res10["avgTimeDiffBetweenSentTransactions"].fillna(datetime.timedelta(0))
res10 = res10.reindex(addresses, fill_value=0.)
res10.index = res10.index.rename("address")
res10 = res10.sort_values(by="address")
res10 = res10.drop(["numberOfTranscationsSent", "seconds_diff"],axis=1)
res10["avgTimeDiffBetweenSentTransactions"] = [ts.total_seconds() for ts in res10["avgTimeDiffBetweenSentTransactions"]]
avg_time_diff_senttx_result_py = res10
avg_time_diff_senttx_result_py

,avgTimeDiffBetweenSentTransactions
address,
exchange_1,840.000000
exchange_10,786.857143
exchange_11,377.538462
exchange_12,878.000000
exchange_13,400.500000
exchange_14,738.857143
exchange_15,568.500000
exchange_16,1296.000000
exchange_17,560.400000


In [361]:
pd.testing.assert_frame_equal(avg_time_diff_senttx_result_sql, avg_time_diff_senttx_result_py)
print("avgTimeDiffBetweenSentTransactions Test succeeded !!")

avgTimeDiffBetweenSentTransactions Test succeeded !!


In [363]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.join(avg_time_diff_senttx_result_sql)
features = features.join(avg_time_diff_receivedtx_result_sql)
features = features.sort_values(by="balance", ascending=False)
features = features.fillna(0.0)
import sys
# addresses that have sent/received only one transaction get the avg time max * 2
features["avgTimeDiffBetweenSentTransactions"] = features["avgTimeDiffBetweenSentTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenSentTransactions"]))
features["avgTimeDiffBetweenReceivedTransactions"] = features["avgTimeDiffBetweenReceivedTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenReceivedTransactions"]))

In [364]:
features = features.reset_index()

In [367]:
features.to_gbq('ethereum_us.features_sampleData', if_exists="replace")

1it [00:04,  4.50s/it]


# Deprecated

In [376]:
tablesToDelete = [
    "token_transfers"
    ,"tokens"
    ,"top40k_19_20_features"
    ,"top40k_19_20_time_rec_tx"
    ,"top40k_week1777_contracts"
,"top40k_week1777_logs"
,"top40k_week1777_token_transfers"
 ,   "top40k_week1777_tokens"
  ,  "top40k_week1777_traces"
   , "traces_sampleData"
    ,"traces_top40k_2019_Q1"
    ,"transactions"
    
]

for t in tablesToDelete:
    client.delete_table("ethereum_us.{}".format(t), not_found_ok=True) 

In [369]:
client.delete_table("ethereum_us.blocks", not_found_ok=False) 

NotFound: 404 DELETE https://www.googleapis.com/bigquery/v2/projects/masterarbeit-245718/datasets/ethereum_us/tables/blocks: Not found: Table masterarbeit-245718:ethereum_us.blocks

In [377]:
tablesToDelete = ["test_1000_address_month_txReceived_txSent"                                                                  
  ,"test_1000_address_txReceived_txSent"                                                                        
  ,"test_1000_address_weiSent_weiReceived"                                                                      
  ,"test_1000_weiSent_weiReceived_withoutTxFees"                                                                
  ,"test_top10_blocks"                                                                                          
  ,"test_top10_contracts"                                                                                       
  ,"test_top10_logs"                                                                                            
  ,"test_top10_token_transfers"                                                                                 
  ,"test_top10_tokens"                                                                                          
  ,"test_top10_traces"                                                                                          
  ,"test_top10_transactions"                                                                                                                                                               
  ,"top40k_week1777_features"                                                                                   
  ,"top40k_week1777_features_avgTimeDiffBetweenReceivedTx"                                                      
  ,"top40k_week1777_features_avgTimeDiffBetweenSentTx"                                                          
  ,"top40k_week1777_features_txSent_txReceived"                                                                 
  ,"top40k_week1777_features_weiSent_weiReceived"           ] 

for t in tablesToDelete:
     client.delete_table("ethereum_us.{}".format(t), not_found_ok=True)  